# Домашнее задание по рекомендательным системам

В данном домашнем задании вам предлагается реализовать User-based рекомендательную систему. Так же требуется реализовать несколько вспомогательных функций, шаблоны которых вы можете найти в `utils.py`.

Требования к выполнению задания:
- Реализация функции из `utils.py` засчитывается, только если пройдены все соответствующие тесты из `test.py`. Запуск тестов: <font color='red'>pytest test.py</font>. Для тестов вам потребуются библиотеки `numpy`, `scipy`, `pytest` и `hypothesis`.
- Плагиат запрещен. Если будет замечено, что часть задания списана, то 0 баллов ставится как списывающему, так и давшему списать.
- Если пользуетесь кодом из открытых источников, то указывайте ссылки, откуда взяли решение. Иначе такой код может быть воспринят как плагиат.
- При выполнении задания нельзя использовать библиотеку `scipy` и функцию `numpy.linalg.norm`

При запуске тестов могут появиться предупреждения: PearsonRConstantInputWarning и PearsonRNearConstantInputWarning. На них можно не обращать внимания.

Возможный максимум баллов за задание: 10 баллов <br>
Дедлайн: ??? <br>
Штраф: ??? - будет ли в курсе штраф? <br>
<br>
Для ускорения проверки, напишите здесь получившееся количество баллов: 10 :-)

<b>Сообщаю, что вспомогательный код брал с ноутбука классной работы</b>

## 1. Метрика сходства
<b>1.1. Реализация метрик (2 балла)</b>

Первое, с чем необходимо разобраться, при реализации User-based подхода, это с метрикой, с помощью которой будет решаться, насколько похожи пользователи. Вам предлагается реализовать 2 метрики: на основе евклидовой метрики и коэффициент корреляции Пирсона. Шаблоны для обоих функций можете найти в `utils.py`. Не забудьте проверить реализацию на тестах.

Евклидова метрика:
\begin{equation}
d(p,q)=\sqrt{(p_1-q_1)^2+(p_2-q_2)^2+\dots+(p_n-q_n)^2} = \sqrt{\sum_{k=1}^n (p_k-q_k)^2}
\end{equation}

В этом случае $d(p, q) \in [0, \infty)$, при этом если $d(p, q) \to 0$, то $sim(p, q) \to 1$. С учетом этого конечная формула будет выглядеть следующим образом:
\begin{equation}
sim(p, q) = \frac{1}{1 + d(p, q)}
\end{equation}
Так же в этой формуле не будет проблем с делением на 0.

Коэффициент корреляции Пирсона:
\begin{equation}
r_{xy} = \frac {\sum_{i=1}^{m} \left( x_i-\bar{x} \right)\left( y_i-\bar{y} \right)}{\sqrt{\sum_{i=1}^{m} \left( x_i-\bar{x} \right)^2 \sum_{i=1}^{m} \left( y_i-\bar{y} \right)^2}}
\end{equation}

<b>1.2. (1 балл)</b>

Рассмотрим пользователей $u$ и $v$. Им соотвествуют векторы $x_u$ и $x_v$, где $x_u[i] = r_{ui}$ и $x_v[i] = r_{vi}$. Из лекции известно, что похожесть между векторами $x_u$ и $x_v$ вычисляются только для тех индексов i, для которых существует и $r_{ui}$, и $r_{vi}$. То есть верно следуюющее:
\begin{equation}
sim(u, v) = sim(x_uI_{uv}, x_vI_{uv}),
\end{equation}
где $I_{uv} = [i | \exists r_{ui} \& \exists r_{vi}]$. При этом если $I_{uv} = \emptyset$, то $sim(u, v) \to -\infty$.

Реализуйте два новых метода, которые переиспользуют написанные вами `euclidean_distance` и `pearson_distance`, добавляющие условия на $x_u$ и $x_v$. Считается, что $x_u[i] = 0$, если $\nexists r_{ui}$. То же верно для $x_v$.

При реализации заданий можно как написать новые функции, так и использовать декораторы.

In [95]:
import numpy as np
from utils import euclidean_similarity, pearson_similarity

In [96]:
def euclidean(x: np.array, y: np.array) -> float :
    common_elements = ((x != 0) & (y != 0))
    return euclidean_similarity(x[common_elements], y[common_elements])

euclidean(np.array([1, 2]), np.array([0, 2]))

1.0

In [97]:
def pearson(x: np.array, y: np.array) -> float :
    common_elements = ((x != 0) & (y != 0))
    return pearson_similarity(x[common_elements], y[common_elements])

pearson(np.array([1, 2, 1, 1, 4, 56, 7, 8, 7]), np.array([0, 2, 3, 5, 8, 1, 3, 1, 5]))

-0.4425807127862795

In [98]:
def inv_euclidean(x: np.array, y: np.array) -> float :
    ed = euclidean(x, y)
    return 1 - ed

def inv_pearson(x: np.array, y: np.array) -> float :
    pd = (pearson(x, y) + 1) / 2
    return 1 - pd

## 2. User-based method
<b>2.1. (3 балла)</b> 

Реализовать User-based подход, реализовав методы класса `UserBasedRecommendation`, основанного на использовании `NearestNeighbors`. В качестве метрики может для нахождения похожих пользователей может быть использована как евклидова метрика, так и коэффициент корреляции Пирсона.

Не забывайте, что `NearestNeighbors` ищет минимум расстояния между элементами, поэтому логично в качестве метрики при инициализации `NearestNeighbors` использовать обратную метрике схожести. То есть такую, что когда $sim(u, v) \to 1$, то $d(u, v) \to 0$. Например: $d(u, v) = 1 - sim(u, v)$

In [99]:
from sklearn.neighbors import NearestNeighbors
from typing import Optional

from collections import Counter


class UserBasedRecommendation:
    
    met = None
    n_rec: int = 5
    al: float = 0.8
    model = None
    data: np.array = None
    
    def __init__(self, metric: str = 'euclidean', n_recommendations: int = 5, alpha: float = 0.8):
        """
        Args:
            metric: name of metric: ['euclidean', 'pearson']
            n_recommendations: number of recommendations. Also can be specified self.make_recommendation
            alpha: similarity threshold: if sim(u, v) > alpha then u and v are similar
        """
        self.met = inv_pearson if metric == 'pearson' else inv_euclidean
        self.model = NearestNeighbors(metric=self.met, n_neighbors=n_recommendations)
        self.al = alpha
        self.n_rec = n_recommendations
    
    def fit(self, X: np.array):
        """
        Args:
            X: matrix N x M where X[u, i] = r_{ui} if r_{ui} exists else X[u, i] = 0
        """
        self.model.fit(X)
        self.data = X

    def find_closest_users(self, user_id: int, n_closest_users: int):
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        closest = self.model.kneighbors(self.data[user_id].reshape(1, -1), n_neighbors=n_closest_users + 1, return_distance=False)[0]
        return closest[closest != user_id]
        

    def make_recommendation(self, user_id: int, n_recommendations: Optional[int] = None):
        """
        Args:
            user_id: user id to whom you want to recommend
            n_recommendations: number of recommendations
        """
        # your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
        if n_recommendations:
            self.n_rec = n_recommendations
        brothers = self.find_closest_users(user_id, n_recommendations)
        recomms = []
        for bro_id in brothers:
            mask = (self.data[bro_id] != 0) & (self.data[user_id] == 0)
            for film in range(len(self.data[bro_id])):
                if film not in recomms and mask[film]:
                    recomms.append(film)
        return recomms[0:self.n_rec]

ubr = UserBasedRecommendation()
ubr.fit(np.array([[1, 2, 3, 0], [8, 7, 4, 0], [3, 0, 4, 5]]))
ubr.make_recommendation(0, 1)

[3]

<b>2.2. (1 балла)</b>

Приведите пример, для которого использование разных метрик будет давать разные рекомендации. Объясните свой пример.

In [100]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
data = np.array([[7, 8, 4, 0], [10, 9, 0, 5], [5, 4, 0, 0]])

ubr_euclidean = UserBasedRecommendation()
ubr_euclidean.fit(data)
print('Euclidean metrics case neighbour:', ubr_euclidean.find_closest_users(2, 1))

ubr_pearson = UserBasedRecommendation(metric='pearson')
ubr_pearson.fit(data)
print('Pearson metrics case neighbour:', ubr_pearson.find_closest_users(2, 1))

Euclidean metrics case neighbour: [0]
Pearson metrics case neighbour: [1]


In [101]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
print('Euclidean metrics case recommendation:', ubr_euclidean.make_recommendation(2, 1))

print('Pearson metrics case recommendation:', ubr_pearson.make_recommendation(2, 1))

Euclidean metrics case recommendation: [2]
Pearson metrics case recommendation: [3]


<b>Объяснение:</b> Для ясности пронумеруем товары с 0. Видно, что третьему пользователю так же как и второму больше понравился товар 1 и чуть меньше понравился товар 2, тогда как первому пользователю оба эти товары не сильно понравились, причем товар 1 ему понравился меньше, чем товар 2, то есть наблюдается обратная завизимость. Но все эти факторы никак не учитываются при использовании евклидовой метрики, поэтому в первом случае алгоритм посчитал, что первый и третий пользователь похожи только по тому, что они ставят оценки примерно в одном и том отрезке: {0, 1}, что является довольно странным предположением. А алгоритм, использующий коэффициент корреляции Пирсона учитывает эти факторы, поэтому находит третьему пользователю соседа со схожими интересами в виде второго пользователя. Отсюда получаем разные рекомендации.

## 3. Оценка качества
<b>3.1. (1 балл)</b>

Реализуйте Average Precision at k и Mean Average Precision at k. Шаблоны можете найти в `utils.py`.
\begin{align*}
AP@K = \frac{1}{m}\sum_{k=1}^K P(k)*rel(k), \\
MAP@K = \frac{1}{|U|}\sum_{u=1}^{|U|}(AP@K)_u
\end{align*}
где $P(k)$ - Precision at k, $rel(k) = 1$, если рекомендация релевантна, иначе $rel(k) = 0$.

## 4. Применение модели
<b>4.1. (2 балла)</b>

Выгрузите датасет `ratings_small.csv`: https://www.kaggle.com/rounakbanik/the-movies-dataset#ratings_small.csv

In [102]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [103]:
data = pd.read_csv('data/ratings_small.csv', index_col=False)
data.shape

(100004, 4)

In [104]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(1, 671, 671)

In [105]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(1, 163949, 9066)

In [106]:
data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


Для простоты работы с данными, измените нумерацию пользователей и фильмов так, чтобы нумерация начиналась с 0 и шла непрерывно.

In [107]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
index_pairs = list(enumerate(data.userId.unique()))
# print(index_pairs)
userId_to_range = {data_id: ind for ind, data_id in index_pairs}
range_to_userId = {ind: data_id for data_id, ind in userId_to_range.items()}

In [108]:
data['userId'] = data['userId'].apply(lambda entry: userId_to_range[entry])

In [109]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
index_pairs = list(enumerate(data.movieId.unique()))
# print(index_pairs)
filmId_to_range = {data_id: ind for ind, data_id in index_pairs}
range_to_filmId = {ind: data_id for data_id, ind in filmId_to_range.items()}

In [110]:
data['movieId'] = data['movieId'].apply(lambda entry: filmId_to_range[entry])

In [111]:
data.userId.min(), data.userId.max(), len(data.userId.unique())

(0, 670, 671)

In [112]:
data.movieId.min(), data.movieId.max(), len(data.movieId.unique())

(0, 9065, 9066)

In [113]:
data.head()

,userId,movieId,rating,timestamp
0,0,0,2.5,1260759144
1,0,1,3.0,1260759179
2,0,2,3.0,1260759182
3,0,3,2.0,1260759185
4,0,4,4.0,1260759205


In [114]:
data.shape

(100004, 4)

Удалим для наиболее активных пользователей 5 оценок

In [120]:
active_users = data.userId.value_counts()[:10].index
test_data = pd.DataFrame([], columns=data.columns)
for user_id in active_users:
    _, test = train_test_split(data[data.userId == user_id], test_size=5, random_state=42)
    test_data = test_data.append(test, ignore_index=True)
    data = data[~((data.userId == user_id) & (data.movieId.isin(test.movieId.values)))]
data.shape, test_data.shape

((99954, 4), (50, 4))

Преобразуем данные в таблицу `X`, с которой может работать `UserBasedRecommendation`, где $X_{ui} = r_{ui}$, если пользователь $u$ поставил оценку фильму $i$, и $X_{ui} = 0$, если пользователь $u$ не проставил оценку фильму $i$.

Вам может пригодиться `csr_matrix`.

In [121]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
import scipy.sparse as scp
def get_sparse(data):
    return scp.coo_matrix(
        (
            data['rating'],  # оценки
            (data['userId'], data['movieId'])  # id пользователей и id фильмов
        ), 
        shape=(len(userId_to_range), len(filmId_to_range))  # размеры матрицы рейтингов
    ).tocsr()

In [124]:
train_rate_matrix = get_sparse(data)
train_rate_matrix

<671x9066 sparse matrix of type '<class 'numpy.float64'>'
	with 99954 stored elements in Compressed Sparse Row format>

In [125]:
test_rate_matrix = get_sparse(test_data)
test_rate_matrix

<671x9066 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>

Для пользователей, у которых были удалены фильмы, найдите топ 100 фильмов, который должен посмотреть каждый из этих пользователей, используя `UserBasedRecommendation`. Не забудьте подобрать параметр alpha.

In [147]:
active_users

Int64Index([546, 563, 623, 14, 72, 451, 467, 379, 310, 29], dtype='int64')

In [148]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
ubr = UserBasedRecommendation()
ubr.fit(train_rate_matrix.todense().A)
recs_to_active = {}
for user in active_users:
    recs_to_active[user] = ubr.make_recommendation(user, 100)

In [155]:
recs_to_active

{546: [128,
  405,
  644,
  804,
  972,
  1012,
  1240,
  1276,
  1289,
  1321,
  1435,
  1512,
  1554,
  1575,
  1597,
  1600,
  1612,
  1613,
  1618,
  1621,
  2144,
  2232,
  2397,
  2456,
  2525,
  2675,
  2749,
  2752,
  3067,
  3124,
  3147,
  3153,
  3179,
  3185,
  3235,
  3249,
  3412,
  3447,
  3904,
  3942,
  3945,
  3957,
  4986,
  5403,
  5404,
  5406,
  5408,
  5820,
  6708,
  6924,
  20,
  93,
  320,
  650,
  651,
  705,
  713,
  721,
  2062,
  2064,
  2066,
  2072,
  2084,
  2093,
  2095,
  2274,
  2275,
  2518,
  3123,
  3401,
  3602,
  347,
  552,
  598,
  627,
  647,
  1061,
  1062,
  1063,
  1072,
  1077,
  1080,
  1083,
  1959,
  2482,
  3075,
  3548,
  4565,
  4681,
  4682,
  143,
  171,
  208,
  319,
  325,
  506,
  662,
  714,
  2075,
  84],
 563: [99,
  137,
  558,
  632,
  848,
  1348,
  1550,
  1565,
  1696,
  1746,
  1776,
  1777,
  2267,
  2608,
  3443,
  3699,
  3749,
  3866,
  4933,
  5075,
  5246,
  6568,
  129,
  373,
  412,
  414,
  478,
  480,
  483,


In [174]:
predicted = []
for user in recs_to_active:
    predicted.append(recs_to_active[user])
len(predicted), len(predicted[0])

(10, 100)

In [181]:
actual = []
m = test_rate_matrix.todense().A
for user in active_users:
    user_line = m[user]
    actual.append(np.where(user_line != 0)[0])
len(actual), len(actual[0])

(10, 5)

Используя метрику `MAP@5`, `MAP@10` и `MAP@100`, определите, насколько эффективна user-based рекомендательная система для данной задачи.

In [157]:
from utils import mapk

In [182]:
# your code (ﾉ>ω<)ﾉ :｡･:*:･ﾟ’★,｡･:*:･ﾟ’☆
mapk(predicted=predicted, actual=actual, k=5)

0.0

In [184]:
mapk(predicted=predicted, actual=actual, k=10)

0.0

In [183]:
mapk(predicted=predicted, actual=actual, k=100)

0.0

Либо я что-то не так сделал, либо рекомендателная система на основе схожести пользователей в данном случае не годится.

Как можно улучшить работу модели?

<b>Ответ:</b> Можно попробовать косинусную метрику схожести, а также использовать разложение SVD для поиска скрытых признаков.